In [ ]:
import sqlite3

In [ ]:
START_YEAR = 2005
CURRENT_SEASON = 2024
COMPLETE_CURRENT_SEASON = False
SPORT_IDS = [1,11,12,13,14,15,16]
MONTHS = [4,5,6,7,8,9] # March folded into April, October into September

In [ ]:
db = sqlite3.connect("BaseballStats.db")
cursor = db.cursor()